Se deben generar 5 robots como Roombas, que limpien una oficina y cada una tiene capacidad de 5 unidades de basura. Al inicio, se debe ingresar un .txt con las dimensiones de la sala (5<=x<=50) y después se dará el mapeo, EJ:


4 4  
S 0 X 3  
1 2 7 X  
2 X 2 2  
0 0 0 P  


6 5  
S 4 0 6 0   
6 X 0 X 4  
0 X 7 X 0  
5 0 0 X 8  
0 0 X 0 0  
0 0 0 0 P  

Los espacios en los que hay una X, son espacios inaccesibles, P es la papelera y S es el punto de partida de los robots. Los robots no pueden conocer el mapa, solo pueden conocer la ubicación de la papelera. Los robots deberán explorar y recoger las unidades de basura que puedan, en caso de requerir ayuda, pueden solicitar ayuda a los demás robots. Cabe destacar que los robots no pueden saber nada del mapa a excepción de lo que se mapea, en caso de que los robots muestren que saben donde hay basura, se penalizarán por 50 puntos.

Los robots son capaces de ver y dar a conocer a los demás robots solo los bloques con los que estén colindando. Solo puede saber a su alrededor si hay obstáculos, otro robot o basura, más no puede saber cuanta basura hay hasta que está en ese espacio.   
EJ:

? ? ? ? ? ? ? ?  
? ? ? B B B ? ?  
? ? ? B R B ? ?  
? ? ? B B X ? ?  
? ? ? ? ? ? ? ?  



Si tienes alguna duda, habla con el profesor o revisa el repo de la clase.


In [297]:
# PARA COLAB SOLAMENTE O ENTORNOS EXTERNOS
# !pip install mesa scikit-learn numpy pandas matplotlib networkx

In [298]:
# Imports
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import SingleGrid
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import heapq
import random
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import pandas as pd
from mesa.datacollection import DataCollector
from IPython.display import HTML
import heapq
import numpy as np
import networkx as nx

# ESTE COMANDO ES PARA LOS PRINTS, PARA VER QUE SALGAN CORRECTO Y ASÍ, ELIMINAR AL FINALIZAR
from IPython.display import display

In [299]:
def leer_mapa(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        mapaREAL = [list(line.replace(" ", "").strip()) for line in lines[1:]]

    filas = len(mapaREAL)
    columnas = len(mapaREAL[0])

    mapaUNKNOWN = [fila[:] for fila in mapaREAL]  # Crear una copia de mapaREAL

    for i in range(filas):
        for j in range(columnas):
            if mapaREAL[i][j] not in ['S', 'P']:
                mapaUNKNOWN[i][j] = '?'
    
    
#    SE IMPRIMEN LOS MAPAS EN TERMINAL
#    print("MapaREAL:")
#    for fila in mapaREAL:
#        print(' '.join(fila))
#    
#    print("MapaUNKNOWN:")
#    for fila in mapaUNKNOWN:
#        print(' '.join(fila))
    
    return mapaREAL, mapaUNKNOWN


#########################################################
# Se comentaron todolo relacionado al grafo para tratar #
# de buscar una solución donde los robots no sepan el   # 
# mapa. Trataré de hacer que 1 robot se rife a explorar #
# todo el mapa y haga el mapeo correspondiente.         #
#########################################################

# def construir_grafo(mapa):
#    grafo = {}
#    filas = len(mapa)
#    columnas = len(mapa[0])
#
#    for i in range(filas):
#        for j in range(columnas):
#            if mapa[i][j] != 'X':
#                nodo = (i, j)
#                vecinos = []
#
#                if i - 1 >= 0 and mapa[i-1][j] != 'X': vecinos.append((i-1, j))
#                if i + 1 < filas and mapa[i+1][j] != 'X': vecinos.append((i+1, j))
#                if j - 1 >= 0 and mapa[i][j-1] != 'X': vecinos.append((i, j-1))
#                if j + 1 < columnas and mapa[i][j+1] != 'X': vecinos.append((i, j+1))
#
#                grafo[nodo] = vecinos
#                
#    
#        # Vamos a dibujar el grafo para entenderlo:
#
#    G = nx.DiGraph()
#
#    # Agrega los nodos al grafo
#    for nodo, vecinos in grafo.items():
#        G.add_node(nodo)
#        for vecino in vecinos:
#            G.add_edge(nodo, vecino)
#            
#    # Se espejea el dibujo para que Rodri no me esté molestando
#    pos = {nodo: (x, -y) for nodo, (x, y) in nx.spring_layout(G).items()}
#    
#    # Dibuja el grafo utilizando matplotlib
#    pos = {nodo: nodo for nodo in G.nodes()}  # Definir la posición de los nodos para que se dibujen en sus coordenadas
#    nx.draw(G, pos, with_labels=True, node_size=1000, font_size=10, node_color='lightblue', font_color='black')
#
#    # Rotar el gráfico
#    plt.xticks(rotation=90)  # Cambia el ángulo de las etiquetas del eje x (en grados)
#
#    # Iguala el aspecto del gráfico para que las proporciones no se distorsionen
#    plt.gca().set_aspect('equal')
#
#    # Muestra el dibujo del grafo
#    plt.show()
#
#    return grafo

In [300]:
# Se definen algoritmos

#########################################################
# Se comentaron todos los algoritmos para tratar de     #
# buscar una solución donde los robots no sepan el mapa # 
# trataré de hacer que 1 robot se rife a explorar todo  #
# el mapa y haga el mapeo correspondiente.              #
#########################################################

# def heuristic(a, b):
#    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Implementación del algoritmo A*
# def a_star_search(graph, start, goal):
#    frontier = []
#    heapq.heappush(frontier, (0, start))
#    came_from = {}
#    cost_so_far = {}
#    came_from[start] = None
#    cost_so_far[start] = 0

#    while frontier:
#        _, current = heapq.heappop(frontier)

#        if current == goal:
#            break

#        for next in graph[current]:
#            new_cost = cost_so_far[current] + 1
#            if next not in cost_so_far or new_cost < cost_so_far[next]:
#                cost_so_far[next] = new_cost
#                priority = new_cost + heuristic(goal, next)
#                heapq.heappush(frontier, (priority, next))
#                came_from[next] = current
# 
#    Reconstruct path
#    current = goal
#    path = []
#    while current != start:
#        path.append(current)
#        current = came_from[current]
#    path.append(start)
#    path.reverse()
#    return path

In [301]:
# Definición de Agentes

# El agente si lo modificaré más de la cuenta, me refiero a que no comentaré lo que no sirva,
# directamente lo borraré para hacer más limpia esta sección.

class Robot(Agent):
    def __init__(self, unique_id, model, inicio, mapaREAL, mapaUNKNOWN):
        super().__init__(unique_id, model)
        self.posicion = inicio
        self.capacidad = 5
        self.memoria = {}
        self.memoria[inicio] = 'S'
        self.basura_no_recogida = None
        self.destinos_pendientes = [0]
        self.mapaREAL = mapaREAL
        self.mapaUNKNOWN = mapaUNKNOWN

    def step(self):
        self.mapeo()
        self.moverse()
        print("Ahora estoy en la posición:")
        print(self.posicion)
        print("Mi memoria es:")
        print(self.memoria)

    def mapeo(self):
        print("Estoy mapeando chucuchú")
        if self.memoria[self.posicion] == 'B':
            print("Exploro detalles VAMOOOS")
            self.explorar_detalle()

        self.memoria[self.posicion] = self.mapaREAL[self.posicion[0]][self.posicion[1]] 

        # Actualizar mapaUNKNOWN con información del mapaREAL
        self.mapaUNKNOWN[self.posicion[0]][self.posicion[1]] = self.mapaREAL[self.posicion[0]][self.posicion[1]]

        # Actualizar memoria con información del entorno
        vecinos = self.model.grid.get_neighborhood(self.posicion, moore=True, include_center=True)
        for vecino in vecinos:
            print(f"Analizando vecino: {vecino}")
            contenido = self.model.mapa[vecino[1]][vecino[0]]  # Corrección aquí
            print(f"Contenido: {contenido}")
            if contenido == 'X':
                self.memoria[vecino] = 'X'
            if contenido.isdigit():
                self.memoria[vecino] = 'B'
            elif contenido == '0':
                self.memoria[vecino] = '0'
            else:
                self.memoria[vecino] = contenido

        # Actualizar celda actual en la grid
        self.model.grid[self.posicion[0]][self.posicion[1]] = self.memoria[self.posicion]


    def explorar_detalle(self):
        contenido = self.model.mapa[self.posicion[0]][self.posicion[1]]
    
        if contenido == 'X':
            print("Se reveló que hay un obstáculo en esta celda.")
            self.memoria[self.posicion] = 'X'
        elif contenido.isdigit():
            print(f"Se reveló que hay {contenido} de basura en esta celda.")
            valor_basura = int(contenido)
            self.memoria[self.posicion] = valor_basura
            self.model.mapa[self.posicion[0]][self.posicion[1]] = str(valor_basura)  # Actualiza también en el mapa
            self.model.grid[self.posicion[0]][self.posicion[1]] = str(valor_basura)  # Actualiza también en la grid
            print(self.memoria)
        else:
            print("No se reveló nada nuevo en esta celda.")
                
    
    def obtener_vecinos_conocidos(self):
        vecinos = self.model.grid.get_neighborhood(self.posicion, moore=True, include_center=False)
        vecinos_conocidos = [vecino for vecino in vecinos if vecino in self.memoria and self.memoria[vecino] != '?']
        return vecinos_conocidos
        
    def moverse(self):
        print("Si me muevo")
        destino = self.destinos_pendientes[0]
        if self.posicion == destino:
            self.destinos_pendientes.pop(0)
            return

        ruta = self.planificar_ruta(destino)
        print("Imprimo la ruta")
        print(ruta)
        siguiente_celda = ruta[0]
        self.posicion = siguiente_celda
        self.model.grid.move_agent(self, siguiente_celda)
        
    def planificar_ruta(self, destino):
        cola = [(self.posicion, [])]  # Cola para el BFS
        print("Imprimo la cola")
        print(cola)
        visitados = set()

        while cola != []:
            celda_actual, ruta_actual = cola.pop(0)
            if celda_actual == destino:
                print("Imprimo la ruta_actual")
                print(ruta_actual)
                return ruta_actual

            if celda_actual in visitados:
                continue

            visitados.add(celda_actual)
            vecinos_conocidos = self.obtener_vecinos_conocidos()
            vecinos_libres = [vecino for vecino in vecinos_conocidos if self.memoria[vecino] != 'X']

            for vecino in vecinos_libres:
                cola.append((vecino, ruta_actual + [vecino]))
                
        return ruta_actual



    
#        # Moverse hacia la basura más cercana
#        if len(camino_corto) > 1:
#            nueva_posicion = camino_corto[1]
#            self.posicion = nueva_posicion
#
#            # Recoger basura
#            if mapa[nueva_posicion[0]][nueva_posicion[1]].isdigit() and int(mapa[nueva_posicion[0]][nueva_posicion[1]]) > 0:
#                basura = min(int(mapa[nueva_posicion[0]][nueva_posicion[1]]), self.capacidad)
#                self.capacidad -= basura
#                mapa[nueva_posicion[0]][nueva_posicion[1]] = str(int(mapa[nueva_posicion[0]][nueva_posicion[1]]) - basura)
#
#                # Si no pudo recoger toda la basura, almacena la posición
#                if int(mapa[nueva_posicion[0]][nueva_posicion[1]]) > 0:
#                    self.basura_no_recogida = nueva_posicion
#
#        # Comunicación entre robots
#        for agent in self.model.schedule.agents:
#            if isinstance(agent, Robot) and agent != self:
#                self.memoria.update(agent.memoria)
#                agent.memoria.update(self.memoria)
#
#                # Comparte la posición de basura no recogida
#                if self.basura_no_recogida:
#                    agent.memoria[self.basura_no_recogida] = 'basura_no_recogida'

In [302]:
# Se define el modelo

class LimpiezaModel(Model):
    def __init__(self, mapa_txt):
        super().__init__(self)
        self.datacollector = DataCollector(
            model_reporters={"Mapa": "mapa"},
            agent_reporters={"Posicion": "posicion"})
        self.num_robots = 1
        self.schedule = RandomActivation(self)
        self.mapa, self.mapaREAL = leer_mapa(mapa_txt)
        self.mapaUNKNOWN = [fila[:] for fila in self.mapa]  # Copia del mapa desconocido
        self.grid = SingleGrid(len(self.mapa), len(self.mapa[0]), torus=False)
        self.bote_basura = ()
        self.basura = ()
        self.obstaculo = ()
        self.mapa, self.mapaREAL = leer_mapa(mapa_txt)
        self.memoria_mapa = {}
        self.todas_celdas_mapeadas = 1

        # self.fig, self.ax = plt.subplots()

        # Creación de robots
        for i in range(self.num_robots):
            robot = Robot(i, self, (0, 0), self.mapaREAL, self.mapaUNKNOWN)
            self.schedule.add(robot)
    
            self.grid.place_agent(robot, (0, 0))

#    def draw_map(self):
#        self.ax.clear()
#        for i, row in enumerate(self.grid):
#            for j, cell in enumerate(row):
#                if cell == 'X':
#                    self.ax.text(j, i, cell, ha='center', va='center', fontsize=60, color='black')
#                elif cell in ['S', 'P']:
#                    self.ax.text(j, i, cell, ha='center', va='center', fontsize=60, color='blue')
#                else:
#                    contenido = self.mapaREAL[i][j]  # Obtener el valor real del mapa
#                    self.ax.text(j, i, contenido, ha='center', va='center', fontsize=60, color='blue')
#        for agent in self.schedule.agents:
#            self.ax.text(agent.posicion[1], agent.posicion[0], 'R', ha='center', va='center', fontsize=40, color='red')            

            
    def step(self):
        print("Paso")   
        self.schedule.step()
    #    self.draw_map()
    # if self.todas_celdas_mapeadas():
    #       self.running = False  # Detener la simulación

In [303]:
# Crear el modelo
model = LimpiezaModel('mapa.txt')

steps = 2
for i in range(steps): 
    model.step()

# Crear una función para visualizar la animación
# def visualizar_animacion(model, steps):
#    fig, ax = plt.subplots()
#
#    def animate(frame):
#        model.step()
#        model.draw_map()
#
#    ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500)
#    animate(0)  # Iniciar la animación
#    plt.show()
#
# visualizar_animacion(model, 100)




Paso
Estoy mapeando chucuchú
Analizando vecino: (0, 0)
Contenido: S
Analizando vecino: (0, 1)
Contenido: 6
Analizando vecino: (1, 0)
Contenido: 4
Analizando vecino: (1, 1)
Contenido: X
Si me muevo
Imprimo la cola
[((0, 0), [])]
Imprimo la ruta
[(1, 0), (1, 0)]
Ahora estoy en la posición:
(1, 0)
Mi memoria es:
{(0, 0): 'S', (0, 1): 'B', (1, 0): 'B', (1, 1): 'X'}
Paso
Estoy mapeando chucuchú
Exploro detalles VAMOOOS
Se reveló que hay 6 de basura en esta celda.
{(0, 0): 'S', (0, 1): 'B', (1, 0): 6, (1, 1): 'X'}
Analizando vecino: (0, 0)
Contenido: S
Analizando vecino: (0, 1)
Contenido: 6
Analizando vecino: (1, 0)
Contenido: 4
Analizando vecino: (1, 1)
Contenido: X
Analizando vecino: (2, 0)
Contenido: 0
Analizando vecino: (2, 1)
Contenido: 0
Si me muevo
Imprimo la cola
[((1, 0), [])]
Imprimo la ruta
[(2, 1), (2, 1)]
Ahora estoy en la posición:
(2, 1)
Mi memoria es:
{(0, 0): 'S', (0, 1): 'B', (1, 0): 'B', (1, 1): 'X', (2, 0): 'B', (2, 1): 'B'}
